In [1]:
import requests as req
from bs4 import BeautifulSoup
import json
import pandas as pd
from standardize import *
from key import api_key
from key import g_api_key
import re
import os
import math
import bokeh as bo
import numpy as np 
from helper import *
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn
import os
import re
from bokeh.io import export_png
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.palettes import Category20
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, HoverTool, Cross
from math import pi

In [2]:
civ_comps = 'https://data.milwaukee.gov/api/3/action/datastore_search?resource_id=cd1ca96a-88dc-4af5-b813-e01bc9e1121f'
cc_df = pull_data(civ_comps,'',api_key)

In [3]:
cc_df.shape

(226, 9)

In [4]:
cc_df.head()

,Category1,Category2,Category3,DateClosed,DateReported,Department,IdentificationNumber,Status,_id
0,No Allegation,None,None,1/23/2018,1/2/2018,Milwaukee Police Department,2018-0001,Closed,1
1,No Allegation,None,None,1/8/2018,1/5/2018,Milwaukee Police Department,2018-0002,Closed,2
2,No Allegation,None,None,1/9/2018,1/8/2018,Milwaukee Police Department,2018-0003,Closed,3
3,Department Procedures,None,None,2/21/2018,1/10/2018,Milwaukee Police Department,2018-0004,Closed,4
4,Department Procedures,None,None,1/16/2018,1/16/2018,Milwaukee Police Department,2018-0005,Closed,5


In [5]:
cc_df.Department.value_counts()

Milwaukee Police Department                               186
NON EMPLOYEE                                               23
Milwaukee Fire Department                                  15
Milwaukee Fire Department, Milwaukee Police Department      1
Voided Incident                                             1
Name: Department, dtype: int64

In [6]:
cc_df.Category1.value_counts()

Department Procedures                84
Department Services                  74
No Allegation                        32
Discourtesy                          22
Use of Force                          6
Unjustified Search                    3
Other                                 2
Voided Incident                       1
Race/Ethnicity based traffic stop     1
#NAME?                                1
Name: Category1, dtype: int64

In [7]:
cc_df.rename(columns={'Category1':'Case_type'}, inplace=True)

In [8]:
civ_comps_dict = 'cc_dict.csv'
ccd_df = pd.read_csv(civ_comps_dict)

In [9]:
ccd_df

,column,type,label,description
0,IdentificationNumber,text,Identification Number,Generic identification number for each complai...
1,Department,text,Department,The department which the complaint is in regar...
2,Status,text,Status,If the investigation is complete the status is...
3,DateReported,text,Date Reported,The date that the complaint was received by th...
4,DateClosed,text,Date Closed,"The date the investigation completed, when app..."
5,Category1,text,Category 1,A general description of the nature of the com...
6,Category2,text,Category 2,Complaints with multiple allegations regarding...
7,Category3,text,Category 3,Complaints with multiple allegations regarding...


In [10]:
add_year_month_day_df(cc_df,'DateReported','Reported')

In [11]:
closed_cases_df = cc_df[cc_df.Status=='Closed']

In [12]:
add_year_month_day_df(closed_cases_df,'DateClosed','Closed')

/Users/andrewyaz/Documents/mke_python_visuals/standardize.py:374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[date_pre+'_dtime'] = df[date_col].apply(get_datetime)
/Users/andrewyaz/Documents/mke_python_visuals/standardize.py:375: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[date_pre+'Year'] = df[date_pre+'_dtime'].apply(get_year)
/Users/andrewyaz/Documents/mke_python_visuals/standardize.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [13]:
closed_cases_df.shape

(219, 17)

In [14]:
closed_cases_df.head()

,Case_type,Category2,Category3,DateClosed,DateReported,Department,IdentificationNumber,Status,_id,Reported_dtime,ReportedYear,ReportedMonth,ReportedDay,Closed_dtime,ClosedYear,ClosedMonth,ClosedDay
0,No Allegation,None,None,1/23/2018,1/2/2018,Milwaukee Police Department,2018-0001,Closed,1,2018-01-02,2018,1,1,2018-01-23,2018,1,1
1,No Allegation,None,None,1/8/2018,1/5/2018,Milwaukee Police Department,2018-0002,Closed,2,2018-01-05,2018,1,4,2018-01-08,2018,1,0
2,No Allegation,None,None,1/9/2018,1/8/2018,Milwaukee Police Department,2018-0003,Closed,3,2018-01-08,2018,1,0,2018-01-09,2018,1,1
3,Department Procedures,None,None,2/21/2018,1/10/2018,Milwaukee Police Department,2018-0004,Closed,4,2018-01-10,2018,1,2,2018-02-21,2018,2,2
4,Department Procedures,None,None,1/16/2018,1/16/2018,Milwaukee Police Department,2018-0005,Closed,5,2018-01-16,2018,1,1,2018-01-16,2018,1,1


In [15]:
closed_cases_df['Date_to_Close'] = closed_cases_df.Closed_dtime-closed_cases_df.Reported_dtime

/Users/andrewyaz/.local/share/virtualenvs/mke_python_visuals-3suzzAfh/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
real_cats = list(closed_cases_df.Case_type.value_counts().to_frame()[:8].reset_index()['index'])
real_cats = [cat for cat in real_cats if cat !='#NAME?']

In [17]:
print(real_cats)

['Department Procedures', 'Department Services', 'No Allegation', 'Discourtesy', 'Use of Force', 'Unjustified Search', 'Voided Incident', 'Other']


In [18]:
clean_closed_df = closed_cases_df[closed_cases_df.Case_type.isin(real_cats)]

In [19]:
clean_closed_df['Days_to_Close'] = clean_closed_df.Date_to_Close.dt.days
type_means_df = clean_closed_df[['Case_type','Days_to_Close']].groupby('Case_type').mean().reset_index()

/Users/andrewyaz/.local/share/virtualenvs/mke_python_visuals-3suzzAfh/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
type_means_df.head()

,Case_type,Days_to_Close
0,Department Procedures,15.617284
1,Department Services,10.246575
2,Discourtesy,4.238095
3,No Allegation,1.125000
4,Other,0.000000


In [21]:
create_bar_plot("Average Time to Close by Complaint Type:", type_means_df,'Case_type', count_type="Mean",
                agg_func_col ='Days_to_Close' , color = False, output=True, use_factor_map=Category20[type_means_df.shape[0]],
                need_counts=False, tooltip_captions = ['Case Type','Mean Time'], tooltip_cols = ['name','cts days'],
               days_of_week=False)

Loading BokehJS ...

In [22]:
print(real_cats)

['Department Procedures', 'Department Services', 'No Allegation', 'Discourtesy', 'Use of Force', 'Unjustified Search', 'Voided Incident', 'Other']


In [24]:
def get_doy(row):
    
    return ['Sunday','Monday','Tuesday','Wednesday','Thursday',\
        'Friday','Saturday'][int(row)]


In [29]:
def create_bar_plot(f_name, df_attrs, attr, color, patient=False,
                    agg_func_col='cts', count_type='Case', count_column_str=False,
                    alt_num_attr=False, output=False, need_counts=True,
                    use_factor_map=False, tooltip_captions=[], tooltip_cols=[],
                    days_of_week=False):
    '''
    use_factor_map = color pallete to use
    '''
    #os.chdir("/Users/ayaspan/Documents/Pathology_Project/cancer_distros/casesets")

    #count_type = 'Case'


#     if alt_num_attr:
#         df_counts = count_by_column(df_attrs,alt_num_attr)
#     else:
#         df_counts = count_by_column(df_attrs,attr,True)

    if need_counts:

        df_attrs = count_by_column(df_attrs, attr)

    #print(df_attrs.head())

    if patient:
        count_type = 'Patient'

    TOOLTIPS = create_tooltip(tooltip_captions, tooltip_cols)


    
    if days_of_week:

        df_attrs[attr] = df_attrs[attr].apply(get_doy)
        
    source = ColumnDataSource(data=dict(attr_x=df_attrs[attr], cts=np.array(df_attrs[agg_func_col]),
                                        name=df_attrs[attr].astype(str)))
    
        
    #print(name[0].upper()+name[1:])

    if not len(TOOLTIPS):
        TOOLTIPS = False

    p = figure(x_range=df_attrs[attr], plot_width=400, plot_height=400, toolbar_location=None,
               title=f_name[0].upper()+f_name[1:],
               tooltips=TOOLTIPS)

    if use_factor_map:
        use_colors = factor_cmap(
            'name', palette=use_factor_map, factors=df_attrs[attr])
    else:
        use_colors = color

    p.vbar(x='name', width=.9, bottom=0, source=source,
           top='cts', fill_color=use_colors)

    p.xaxis.major_label_orientation = pi/2.7

    if output:
        output_notebook()
        show(p)
        f_name = f_name+"_"+count_type+"_"+"plot.html"
        output_file(filename=f_name)
    else:
        export_png(p, filename=f_name+"_"+count_type+"_"+"plot.png")

In [37]:
#import math


closed_cases_df['day_str'] = create_year_str(closed_cases_df, 'ReportedDay')



for cat in real_cats:
    sub_df = closed_cases_df[closed_cases_df['Case_type']==cat]
    #print(sub_df)
    #print(camel_to_snake(cat).replace('/',"_"))
    name = camel_to_snake(cat).replace('/',"_")
    #print(name)
    create_bar_plot(name+" Cases by Day of Week:", sub_df,'day_str', color = False, use_factor_map=Category20[len(real_cats)],
                need_counts=True, tooltip_captions = ['Day','Cases'], tooltip_cols = ['name','cts'],
                   days_of_week=True)


/Users/andrewyaz/.local/share/virtualenvs/mke_python_visuals-3suzzAfh/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [31]:
race = real_cats[-2]